# Optimize a portfolio
ML for trading Udacity Course exercise

Analyze a portfolio.
http://quantsoftware.gatech.edu/Optimize_something

A transcription of the Udacity Course lectures can be find on https://docs.google.com/document/d/1ELqlnuTSdc9-MDHOkV0uvSY4RmI1eslyQlU9DgOY_jc/edit?usp=sharing

Kairoart 2018
"""


## Overview

In this project you will use what you learned about optimizers to optimize a portfolio. That means that you will find how much of a portfolio's funds should be allocated to each stock so as to optimize it's performance. We can optimize for many different metrics. In previous versions of this assignment, we tried to maximize Sharpe Ratio. We're changing it this time to define "optimal" as minimum volatility.

You will leverage the functions you created in the "asess_portfolio" project that assessed the value of a portfolio with a given set of allocations.

## Task
Implement a Python function named optimize_portfolio() that can find the optimal allocations for a given set of stocks. You should **optimize for minimum volatility** (i.e., standard deviation of daily return).

The function should accept as input a list of symbols as well as start and end dates and return a list of floats (as a one-dimensional numpy array) that represents the allocations to each of the equities. You should take advantage of routines developed in the portfolio analysis project to compute daily portfolio value and statistics. You should cut-and-paste your code for the functions that did this from the last project into optimization.py.

You are given the following inputs for optimizing a portfolio:

* A date range to select the historical data to use (specified by a start and end date)
* Symbols for equities (e.g., GOOG, AAPL, GLD, XOM). Note: You should support any symbol in the data directory.

You should implement the following API EXACTLY.

import datetime as dt
allocs, cr, adr, sddr, sr = \
    optimize_portfolio(sd=dt.datetime(2008,1,1), ed=dt.datetime(2009,1,1), \
    syms=['GOOG','AAPL','GLD','XOM'], gen_plot=False)


### Inputs

* sd: A datetime object that represents the start date
* ed: A datetime object that represents the end date
* syms: A list of symbols that make up the portfolio (note that your code should support any symbol in the data directory)
* gen_plot: If True, create a plot named plot.png

### Output

* allocs: A 1-d Numpy ndarray of allocations to the stocks. All the allocations must be between 0.0 and 1.0 and they must sum to 1.0.
* cr: Cumulative return
* adr: Average daily return
* sddr: Standard deviation of daily return
* sr: Sharpe ratio

## Goal

Your goal is to find allocations to the symbols that optimize the criteria given above. Assume 252 trading days in a year and a risk free return of 0.0 per day.

## Import libraries

In [8]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy
import os
import matplotlib.pyplot as plt
import scipy.optimize as sco

# Add parent directory PATH for looking for modules,
import sys
sys.path.insert(0,'..')
from util import get_data, plot_data

## Define functions

### Find optimal allocations

* We have to provide a function to minimize f(x) that takes in x. (x = allocs)
* An initial guess for that x. 
* Call the optimizer and let it run.


In [37]:
# Define input parameters
start_date = dt.datetime(2010,1,1)
end_date = dt.datetime(2010,12,31)
syms = ['GOOG', 'AAPL', 'GLD', 'XOM']
sc=1000000

# Read in adjusted closing prices for given symbols, date range
dates = pd.date_range(start_date, end_date)
prices_all = get_data(syms, dates)  # automatically adds SPY
prices = prices_all[syms]  # only portfolio symbols



# Number optimal allocations
noa = len(prices.columns)
# Using shift with positive integer shifts rows value downwards
rets = np.log(prices / prices.shift(1))
#print(rets)

# Expected portfolio returns
prets = [0, 0, 0, 0]
# Expected portfolio volatility    
pvols = [0, 0, 0, 0]


for p in range (2500):
    allocs = np.random.random(noa)
    allocs /= np.sum(allocs)
    prets.append(np.sum(rets.mean() * allocs) * 252)
    pvols.append(np.sqrt(np.dot(allocs.T, np.dot(rets.cov() * 252, allocs))))

def statistics(allocs):
        ''' Returns portfolio statistics.
        Parameters
        ==========
        @allocs : <array-like> weights for different securities in portfolio
        
        Returns
        =======
        @pret : <float> expected portfolio return
        @pvol : <float> expected portfolio volatility
        @pret / pvol : <float> Sharpe ratio for rf=0
        '''
        allocs = np.array(allocs)
        pret = np.sum(rets.mean() * allocs) * 252
        pvol = np.sqrt(np.dot(allocs.T, np.dot(rets.cov() * 252, allocs)))
        return np.array([pret, pvol, pret / pvol]) 


def min_func_volatility(allocs):
        return -statistics(allocs)[1]

# Constrains: sum of our allocations to add up to one
# https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.minimize.html
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Ranges: for each of the various allocations, it's only worth looking at values between 0 and 1
bnds = tuple((0, 1) for x in range(noa))

# Minimization of scalar function of one or more variables.
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
opts = sco.minimize(min_func_volatility, noa * [1. / noa,], method='SLSQP', bounds=bnds, constraints=cons)
#print(opts)
allocs = opts['x']
#print(allocs)




                GOOG  AAPL           GLD  XOM
2010-01-04  1.000000   0.0  6.106227e-16  0.0
2010-01-05  0.995596   0.0  6.100665e-16  0.0
2010-01-06  0.970499   0.0  6.201324e-16  0.0
2010-01-07  0.947906   0.0  6.162951e-16  0.0
2010-01-08  0.960542   0.0  6.193538e-16  0.0
2010-01-11  0.959091   0.0  6.275844e-16  0.0
2010-01-12  0.942130   0.0  6.144599e-16  0.0
2010-01-13  0.936721   0.0  6.202992e-16  0.0
2010-01-14  0.941125   0.0  6.230242e-16  0.0
2010-01-15  0.925409   0.0  6.165176e-16  0.0
2010-01-19  0.937567   0.0  6.201880e-16  0.0
2010-01-20  0.926063   0.0  6.058400e-16  0.0
2010-01-21  0.930164   0.0  5.971089e-16  0.0
2010-01-22  0.877559   0.0  5.959966e-16  0.0
2010-01-25  0.861588   0.0  5.977206e-16  0.0
2010-01-26  0.865449   0.0  5.981655e-16  0.0
2010-01-27  0.864938   0.0  5.924375e-16  0.0
2010-01-28  0.852477   0.0  5.921594e-16  0.0
2010-01-29  0.845536   0.0  5.892676e-16  0.0
2010-02-01  0.850451   0.0  6.025589e-16  0.0
2010-02-02  0.847419   0.0  6.0689

In [19]:
def find_optimal_allocations(prices):

    # Number optimal allocations
    noa = len(prices.columns)
    # Using shift with positive integer shifts rows value downwards
    rets = np.log(prices / prices.shift(1))
    #print rets

    # Expected portfolio returns
    prets = [0, 0, 0, 0]
    # Expected portfolio volatility    
    pvols = [0, 0, 0, 0]
    
    # Optimizer
    for p in range (2500):
        allocs = np.random.random(noa)
        allocs /= np.sum(allocs)
        prets.append(np.sum(rets.mean() * allocs) * 252)
        pvols.append(np.sqrt(np.dot(allocs.T, np.dot(rets.cov() * 252, allocs))))
    
    
    prets = np.array(prets)
    pvols = np.array(pvols)

    def statistics(allocs):
        ''' Returns portfolio statistics.
        Parameters
        ==========
        @allocs : <array-like> weights for different securities in portfolio
        
        Returns
        =======
        @pret : <float> expected portfolio return
        @pvol : <float> expected portfolio volatility
        @pret / pvol : <float> Sharpe ratio for rf=0
        '''
        allocs = np.array(allocs)
        pret = np.sum(rets.mean() * allocs) * 252
        pvol = np.sqrt(np.dot(allocs.T, np.dot(rets.cov() * 252, allocs)))
        return np.array([pret, pvol, pret / pvol])


    def min_func_volatility(allocs):
        return -statistics(allocs)[1]

    # Constrains: sum of our allocations to add up to one
    # https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.minimize.html
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    # Ranges: for each of the various allocations, it's only worth looking at values between 0 and 1
    bnds = tuple((0, 1) for x in range(noa))

    # Minimization of scalar function of one or more variables.
    # 
    opts = sco.minimize(min_func_volatility, noa * [1. / noa,], method='SLSQP', bounds=bnds, constraints=cons)

    allocs = opts['x']
    return allocs



In [32]:
# This is the function that will be tested by the autograder
# The student must update this code to properly implement the functionality
def optimize_portfolio(sd = dt.datetime(2008,1,1), ed = dt.datetime(2009,1,1), \
    syms = ['GOOG','AAPL','GLD','XOM'], sv=1000000, gen_plot=False):

    # Read in adjusted closing prices for given symbols, date range
    dates = pd.date_range(sd, ed)
    prices_all = get_data(syms, dates)  # automatically adds SPY
    prices = prices_all[syms]  # only portfolio symbols
    prices_SPY = prices_all['SPY']  # only SPY, for comparison later

    
    # 1. Calculate alloced
    noa = len(symbols)
    allocs = np.random.random(noa)
    allocs /= np.sum(allocs)
    allocs = find_optimal_allocations(prices) #optimize allocations
    allocs = alloced / np.sum(allocs)  # normalize allocations, if they don't sum to 1.0

    # Get daily portfolio value    
    normed = prices / prices.iloc[0]
    alloced = normed * allocs
    pos_vals = alloced * sv
    pos_vals = np.sum(pos_vals,axis=1)
    port_val = pos_vals

    # Get portfolio statistics (note: std_daily_ret = volatility)
    # 1. Cumulative return
    cr = (port_val[-1] / port_val[0] -1)
    
    # 2. Averrage daily returns
    adr = daily_rets.mean()
    # 3. Standard deviation of daily returns
    sddr = daily_rets.std()
    # 4. Sharpe ratio
    sr = adr / sddr
    k = math.sqrt(252)
    sr = k * sr
    
    # Compare daily portfolio value with SPY using a normalized plot
    if gen_plot:
        # Normalize port_val
        x = port_val[0]
        port_val = port_val.divide(x, fill_value=1)

        # Normalize SPY
        x = prices_SPY[0]
        prices_SPY = prices_SPY.divide(x, fill_value=1)

        df_temp = pd.concat([port_val, prices_SPY], 
                             keys=['Portfolio', 'SPY'], axis=1)
        df_temp.plot()
        
    #print(df_temp)
    

    return cr, adr, sddr, sr

def test_code():
    # This code WILL NOT be tested by the auto grader
    # It is only here to help you set up and test your code

    # Define input parameters
    # Note that ALL of these values will be set to different values by
    # the autograder!
    start_date = dt.datetime(2010,1,1)
    end_date = dt.datetime(2010,12,31)
    symbols = ['GOOG', 'AAPL', 'GLD', 'XOM']
    allocations = [0.2, 0.3, 0.4, 0.1]
    start_val = 1000000  
    risk_free_rate = 0.0
    sample_freq = 252

    # Assess the portfolio
    allocs, cr, adr, sddr, sr = optimize_portfolio(sd = start_date, ed = end_date,\
        syms = symbols, \
        gen_plot = True)

    # Print statistics
    print("Start Date:", sd)
    print("End Date:", ed)
    print("Symbols:", syms)
    print("Allocations:", allocs)
    print("Sharpe Ratio:", sr)
    print("Volatility (stdev of daily returns):", sddr)
    print("Average Daily Return:", adr)
    print("Cumulative Return:", cr)

## Testing

In [33]:
test_code()

0.16526140901332545


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices